In [ ]:
#!pip install ruamel.yaml
#!pip install --upgrade jinja2
#!pip install sweetviz

In [ ]:
import pandas as pd
import numpy as np
#import pandas_profiling

In [ ]:
df = pd.read_csv('allos_cluster_data.csv')

In [ ]:
df.info()

In [ ]:
df = df.drop(['Unnamed: 13', 'Unnamed: 14','Unnamed: 15'], axis=1)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df.info()

In [ ]:
# import sweetviz as sv
# report = sv.analyze(df)
# report.show_html('output.html')

In [ ]:
#!pip install dtale

In [ ]:
# import dtale

# # Assuming df is your DataFrame
# # df = pd.read_csv('your_data.csv')

# d = dtale.show(df)
# d.open_browser()

In [ ]:
#d.open_browser()

In [ ]:
df['YearMonth'] = df['Date'].dt.to_period('M')  # Cria uma nova coluna com o ano e o mês

# Agrupa por 'Host name' e 'YearMonth' e conta os meses únicos para cada 'Host name'
months_count = df.groupby('Host name')['YearMonth'].nunique()

#print(months_count)

In [ ]:
df['YearMonth'] = df['Date'].dt.to_period('M')  # Cria uma nova coluna com o ano e o mês

# Cria a tabela cruzada
cross_tab = pd.crosstab(df['Host name'], df['YearMonth'])

#print(cross_tab)

In [ ]:
#cross_tab

In [ ]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek

In [ ]:
df['Month_sin'] = np.sin((df['Date'].dt.month-1)*(2.*np.pi/12))
df['Month_cos'] = np.cos((df['Date'].dt.month-1)*(2.*np.pi/12))
df['Day_sin'] = np.sin((df['Date'].dt.day-1)*(2.*np.pi/30.))
df['Day_cos'] = np.cos((df['Date'].dt.day-1)*(2.*np.pi/30.))

In [ ]:
reference_date = pd.Timestamp('2000-01-01')
df['TimeDifference'] = (df['Date'] - reference_date).dt.days

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
# df_encoded = df[['Host name', 'Branding interest', 'User gender',
#        'City', 'Event name', 'Device category', 'Active users',
#        'Engaged sessions', 'Sessions', 'New users', 'Event count', 'Month_sin', 'Month_cos',
#        'Day_sin', 'Day_cos']]

# df_encoded.info()

In [ ]:
df['Host name'].value_counts()

In [ ]:
hosts_to_remove = ['compras.cariocashopping.com.br', 'compras.shoppingleblon.com.br', 'boulevardcampos.com.br']
df = df[~df['Host name'].isin(hosts_to_remove)]

In [ ]:
df['Host name'] = df['Host name'].replace('www.montesclarosshopping.com.br', 'montesclarosshopping.com.br')

In [ ]:
df['Host name'].value_counts()

In [ ]:
df['Host name'].value_counts().sum()

In [ ]:
df['Host name'].nunique()

In [ ]:
df.info()

In [ ]:
# Get the names of columns of type 'object'
object_columns = df.select_dtypes(include=['object']).columns

# Create a temporary 'combined' column that combines the object columns without filling in the missing values
df['combined_temp'] = df[object_columns].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

# Determine the maximum length of the underscore-separated values in the temporary 'combined' column
max_len = df['combined_temp'].str.split('_').apply(len).max()

# Define a function that joins a row into a string with exactly max_len underscore-separated values
def join_row(row):
    # Convert the row to a list
    values = list(row.values.astype(str))
    # If there are fewer than max_len values, append placeholders ('NA') to the list
    while len(values) < max_len:
        values.append('NA')
    # If there are more than max_len values, remove the extra values
    values = values[:max_len]
    # Join the values into a string with underscore separators
    return '_'.join(values)

# Create the 'combined' column with the missing values filled in
df['combined'] = df[object_columns].apply(join_row, axis=1)

# Drop the temporary 'combined' column
df.drop(columns=['combined_temp'], inplace=True)

# Create a pivot table with the new column as the index
pivot_table = df.pivot_table(index='combined')

pivot_table.info()

In [ ]:
object_columns

In [ ]:
pivot_table.index

In [ ]:
pivot_table['Year'] = pivot_table['Year'].astype(int)

In [ ]:
pivot_table.head()

In [ ]:
pivot_table.info()

In [ ]:
from sklearn.impute import KNNImputer

# Assuming df is your DataFrame
imputer = KNNImputer(n_neighbors=4)

# Fit on the dataset and transform it
df_imputed = imputer.fit_transform(pivot_table)

# Convert the result back to a DataFrame
df_imputed = pd.DataFrame(df_imputed, columns=pivot_table.columns, index=pivot_table.index)

In [ ]:
df_imputed

In [ ]:
df_imputed.info()

In [ ]:
from sklearn.impute import SimpleImputer
#import numpy as np
#import pandas as pd

# Convert the numpy array back to a DataFrame
df_imputed = pd.DataFrame(df_imputed)

# Replace 0s with NaNs
df_imputed.replace(0, np.nan, inplace=True)

# Store column and index information
columns = df_imputed.columns
index = df_imputed.index

# Create an instance of SimpleImputer with strategy as 'mean'
imputer = SimpleImputer(strategy='mean')

# Fit on the dataset and transform it
df_imputed = imputer.fit_transform(df_imputed)

# Convert the result back to a DataFrame using the stored column and index information
df_imputed = pd.DataFrame(df_imputed, columns=columns, index=index)

In [ ]:
print(pivot_table.info())


In [ ]:
print (df_imputed.info())

In [ ]:
null_count = df_imputed['Active users'].isnull().sum()
null_count

In [ ]:
df_imputed['Active users'].value_counts()

In [ ]:
print(df_imputed['Active users'].max())
print(df_imputed['Active users'].min())

In [ ]:
df_imputed['Active users'].value_counts(dropna=False)

In [ ]:
df_imputed['Active users'].isna().sum()

In [ ]:
df_imputed.columns

In [ ]:
df_imputed['Active users'].isna().sum()

In [ ]:
df_imputed.columns

In [ ]:
df_imputed.info()

In [ ]:
# Reset the index
df_imputed.reset_index(inplace=True)

# Print the columns of the DataFrame
print(df_imputed.columns)

df_imputed.info()

In [ ]:
df_imputed.info()

In [ ]:
# Split the 'combined' column into multiple columns
df_split = df_imputed['combined'].str.split('_', n=len(object_columns)-1, expand=True)

# Rename the new columns to match the original object columns
df_split.columns = object_columns

# Drop the 'combined' column from the original DataFrame
df_imputed.drop(columns=['combined'], inplace=True)

# Concatenate the original DataFrame with the new DataFrame
df = pd.concat([df_imputed, df_split], axis=1)

In [ ]:
df.info()

In [ ]:

df['Month'] = df['Month'].astype(float).astype(int)

In [ ]:
df['Month'] = df['Month'].astype(int)

In [ ]:
# Convert 'year' and 'Month' to string
df['Year'] = df['Year'].astype(str)
df['Month'] = df['Month'].astype(str)

# Combine 'year' and 'Month' columns
df['year_month'] = df['Year'] + '_' + df['Month']

In [ ]:
df['year_month'].value_counts() 

In [ ]:
df['year_month'] = df['year_month'].str.replace('.0_', '-')

In [ ]:
df['year_month'].value_counts() 

In [ ]:
# Cria a tabela cruzada
cross_tab_encod = pd.crosstab(df['Host name'], df['year_month'])
#cross_tab_encod

In [ ]:
pivot_table = df.pivot_table(index='Host name', columns='year_month', values='Active users', aggfunc='count')
#pivot_table

In [ ]:
pivot_table

In [ ]:
df.columns

In [ ]:
# Agrupa por 'Host name' e 'YearMonth' e conta os meses únicos para cada 'Host name'
months_count = df.groupby('Host name')['year_month'].nunique()

print(months_count)

In [ ]:
df.info()

In [ ]:
df1 = df.drop(['year_month', 'TimeDifference', 'Day', 'DayOfWeek', 'Month', 'Year'], axis=1)

In [ ]:
df1.columns

In [ ]:
'''
Este código primeiro cria uma cópia do df1 chamada df1_1 e aplica o LabelEncoder a cada coluna de objeto. 
Em seguida, ele cria outra cópia do df1 chamada df2 e aplica o OneHotEncoder a cada coluna de objeto. 
O argumento drop='first' no OneHotEncoder evita a multicolinearidade removendo a primeira coluna dummy 
para cada variável categórica.

'''

from sklearn.preprocessing import LabelEncoder
import pandas as pd

#  uma cópia do df1 
df1_1 = df1.copy()

#  um LabelEncoder
le = LabelEncoder()

# Aplique o LabelEncoder a cada coluna de objeto
for col in df1_1.select_dtypes(include=['object']).columns:
    df1_1[col] = le.fit_transform(df1_1[col])

#  uma cópia do df1 para o OneHotEncoder
df_2 = df1.copy()

# Aplique o OneHotEncoder a cada coluna de objeto
df_2 = pd.get_dummies(df_2, drop_first=True)

In [ ]:
df1_1.head()


In [ ]:
print(df1_1.info())

In [ ]:
# Agrupe por 'Host name' e calcule a média de cada grupo
df_grouped_1 = df1_1.groupby('Host name').mean()

# Reset the index
df_grouped_1 = df_grouped_1.reset_index()

df_grouped_1.info()

In [ ]:
import os
os.environ['OMP_NUM_THREADS'] = '1'

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

# Selecione apenas as colunas numéricas e preencha os valores NA
df_numeric = df_grouped_1.select_dtypes(include=[np.number])
df_numeric = df_numeric.fillna(df_numeric.mean())

# Aplique PCA
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_numeric)

# Aplique KMeans ao resultado do PCA
# O número de clusters é 3
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_pca)

# Adicione os rótulos de cluster ao DataFrame
df_grouped_1['cluster'] = kmeans.labels_

# Calcule o Silhouette Score
score = silhouette_score(df_pca, kmeans.labels_)
print("Silhouette Score: ", score)

In [ ]:
import plotly.express as px

# Crie um DataFrame com os resultados do PCA e os rótulos de cluster
df_plot = pd.DataFrame(df_pca, columns=['PC1', 'PC2'])
df_plot['cluster'] = kmeans.labels_
df_plot['Host name'] = df_grouped_1['Host name']

# Crie um gráfico de dispersão dos resultados do PCA, colorido pelos rótulos de cluster
fig = px.scatter(df_plot, x='PC1', y='PC2', color='cluster', hover_data=['Host name'])

# Mostre o gráfico
fig.show()

In [ ]:
df_grouped_1[['Host name', 'cluster']].nunique()

In [ ]:
df_grouped_1[['Host name', 'cluster']]

In [ ]:
df_grouped_1_ = df_grouped_1[['Host name', 'cluster']]

In [ ]:
df_grouped_1_.info()

In [ ]:
df1_1.info()

In [ ]:
# Mescla df1_1 e df_grouped_1_ com base na coluna 'Host name'
merged_df = df1_1.merge(df_grouped_1_, on='Host name', how='left')

In [ ]:
merged_df.info()

In [ ]:
cluster_lencode = merged_df[['Host name', 'cluster']]

In [ ]:
# Renomeia a coluna 'Host name' para 'Host_name_le' in-place
cluster_lencode.rename(columns={'Host name': 'Host_name_le'}, inplace=True)

In [ ]:
# Concatene df e cluster_lencode
cluster_labelencode = pd.concat([df, cluster_lencode], axis=1)
cluster_labelencode.info()

In [ ]:
# Encontre as colunas que contêm '_cos' ou '_sin' no nome
cols_to_drop = cluster_labelencode.filter(regex='_cos|_sin').columns

# Remova essas colunas
cluster_labelencode = cluster_labelencode.drop(columns=cols_to_drop)

In [ ]:
# Remova a coluna 'TimeDifference' in-place
cluster_labelencode.drop(columns='TimeDifference', inplace=True)

In [ ]:
cluster_labelencode.columns

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Encontre as colunas numéricas
numeric_cols = cluster_labelencode.select_dtypes(include=['int64', 'float64']).columns

# Crie um boxplot para cada coluna numérica
for col in numeric_cols:
    fig = px.box(cluster_labelencode, x='Host name', y=col, color='cluster', title=f'Boxplot of {col} by Host name and cluster',
                 color_discrete_sequence=px.colors.qualitative.Pastel)  # Use a sequência de cores 'Pastel'
    fig.update_layout(
        xaxis={'categoryorder':'total descending'},
        autosize=False,
        width=800,
        height=600,
        plot_bgcolor='rgba(0,0,0,0.02)'  # Fundo transparente
    )
    fig.show()

In [ ]:
cluster_labelencode.columns

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Encontre as colunas numéricas
numeric_cols = cluster_labelencode.select_dtypes(include=['int64', 'float64']).columns

# Encontre os clusters únicos
unique_clusters = cluster_labelencode['cluster'].unique()

# Crie um gráfico de barras para cada coluna numérica e cada cluster
for cluster in unique_clusters:
    for col in numeric_cols:
        # Filtrar o dataframe para o cluster atual e calcular a média
        df_filtered = cluster_labelencode[cluster_labelencode['cluster'] == cluster]
        df_filtered = df_filtered.groupby('Host name')[col].mean().reset_index()
        
        fig = px.bar(df_filtered, x='Host name', y=col, title=f'Bar plot of average {col} by Host name and cluster {cluster}',
                     color_discrete_sequence=px.colors.qualitative.Pastel)  # Use a sequência de cores 'Pastel'
        fig.update_layout(
            xaxis={'categoryorder':'total descending'},
            autosize=False,
            width=800,
            height=600,
            plot_bgcolor='rgba(0,0,0,0.02)',  # Fundo quase transparente
            showlegend=False  # Esconde a legenda
        )
        fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Encontre as colunas numéricas
numeric_cols = cluster_labelencode.select_dtypes(include=['int64', 'float64']).columns

# Encontre os clusters únicos
unique_clusters = cluster_labelencode['cluster'].unique()

# Crie um gráfico de barras para cada coluna numérica e cada cluster
for cluster in unique_clusters:
    for col in numeric_cols:
        # Filtrar o dataframe para o cluster atual e calcular a média
        df_filtered = cluster_labelencode[cluster_labelencode['cluster'] == cluster]
        df_filtered = df_filtered.groupby(['Host name', 'year_month'])[col].mean().reset_index()
        
        fig = px.bar(df_filtered, x='Host name', y=col, color='year_month', title=f'Bar plot of average {col} by Host name, year_month and cluster {cluster}',
                     color_discrete_sequence=px.colors.qualitative.Pastel)  # Use a sequência de cores 'Pastel'
        fig.update_layout(
            xaxis={'categoryorder':'total descending'},
            autosize=False,
            width=800,
            height=600,
            plot_bgcolor='rgba(0,0,0,0.02)',  # Fundo quase transparente
            showlegend=True  # Mostra a legenda
        )
        fig.show()

In [ ]:
!pip install pdfkit
!pip install PyPDF2

In [ ]:
config = pdfkit.configuration(wkhtmltopdf='C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe')  # Use o caminho correto para o seu sistema
pdfkit.from_file(html_file, pdf_file, configuration=config)

In [ ]:
import os
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objects as go

# Encontre as colunas numéricas
numeric_cols = cluster_labelencode.select_dtypes(include=['int64', 'float64']).columns

# Get the path to your home directory
home_dir = os.path.expanduser("~")

# Gráficos 1
for i, col in enumerate(numeric_cols):
    # Create a path for the HTML file in your home directory
    html_file = os.path.join(home_dir, f'graph1_{i}.html')

    fig = px.box(cluster_labelencode, x='Host name', y=col, color='cluster', title=f'Boxplot of {col} by Host name and cluster',
                 color_discrete_sequence=px.colors.qualitative.Pastel)  # Use a sequência de cores 'Pastel'
    fig.update_layout(
        xaxis={'categoryorder':'total descending'},
        autosize=False,
        width=800,
        height=600,
        plot_bgcolor='rgba(0,0,0,0.02)'  # Fundo transparente
    )
    pyo.plot(fig, filename=html_file, auto_open=False)

# Gráficos 2
unique_clusters = cluster_labelencode['cluster'].unique()
for i, cluster in enumerate(unique_clusters):
    for j, col in enumerate(numeric_cols):
        # Create a path for the HTML file in your home directory
        html_file = os.path.join(home_dir, f'graph2_{i}_{j}.html')

        df_filtered = cluster_labelencode[cluster_labelencode['cluster'] == cluster]
        df_filtered = df_filtered.groupby('Host name')[col].mean().reset_index()
        
        fig = px.bar(df_filtered, x='Host name', y=col, title=f'Bar plot of average {col} by Host name and cluster {cluster}',
                     color_discrete_sequence=px.colors.qualitative.Pastel)  # Use a sequência de cores 'Pastel'
        fig.update_layout(
            xaxis={'categoryorder':'total descending'},
            autosize=False,
            width=800,
            height=600,
            plot_bgcolor='rgba(0,0,0,0.02)',  # Fundo quase transparente
            showlegend=False  # Esconde a legenda
        )
        pyo.plot(fig, filename=html_file, auto_open=False)

# Gráficos 3
for i, cluster in enumerate(unique_clusters):
    for j, col in enumerate(numeric_cols):
        # Create a path for the HTML file in your home directory
        html_file = os.path.join(home_dir, f'graph3_{i}_{j}.html')

        df_filtered = cluster_labelencode[cluster_labelencode['cluster'] == cluster]
        df_filtered = df_filtered.groupby(['Host name', 'year_month'])[col].mean().reset_index()
        
        fig = px.bar(df_filtered, x='Host name', y=col, color='year_month', title=f'Bar plot of average {col} by Host name, year_month and cluster {cluster}',
                     color_discrete_sequence=px.colors.qualitative.Pastel)  # Use a sequência de cores 'Pastel'
        fig.update_layout(
            xaxis={'categoryorder':'total descending'},
            autosize=False,
            width=800,
            height=600,
            plot_bgcolor='rgba(0,0,0,0.02)',  # Fundo quase transparente
            showlegend=True  # Mostra a legenda
        )
        pyo.plot(fig, filename=html_file, auto_open=False)